In [ ]:
!pip install pyodbc
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import os
import pyodbc
import json

In [ ]:
SERVER = 'sql-2dkxelm5py5us.database.windows.net'
DATABASE = 'sql-2dkxelm5py5us'
USERNAME = 'sqladmin'
PASSWORD = ''
connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

#https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16

In [ ]:
# Create connection to database
conn = pyodbc.connect(connection_string)

In [ ]:
tables = []

with conn:
    cursor = conn.cursor()
    for row in cursor.tables():
        type = row.table_type
        name = row.table_name

        if type != 'TABLE':
            continue

        tables.append(name)

cursor.close()

In [ ]:
os.makedirs('tables', exist_ok=True)

In [ ]:
# Print the table types and names
with conn:
    for name in tables:
        cursor = conn.cursor()

        columns = []

        for col in cursor.columns(table=name):
            col_name = col.column_name
            col_type = col.type_name
            columns.append({
                'name' : col_name,
                'description' : col_name})

        jTable = {
            'table': name,
            'datasource' : SERVER,
            'description' : name,
            'columns': columns
        }

        #save to json file
        with open(f'tables/{name}.json', 'w') as f:
            f.write(json.dumps(jTable))
cursor.close()

In [ ]:
conn.close()